# Getting started

In [1]:
import numpy as np
from roseau.load_flow import *

## Creating a network

An electrical network can be built by assembling basic elements provided in `roseau.load_flow.models`. This module
contains the following elements:

* Buses:
  * `Bus`: a class for bus
  * `VoltageSource`: a class to depict a voltage source
* Lines/Switches:
  * `LineCharacteristics`: an object to contain the characteristics of a line
  * `ShuntLine`: a line with a shunt admittance
  * `SimplifiedLine`: a line with only a line impedance
  * `Switch`: a basic switch element
* Transformers:
  * `TransformerCharacteristics`: an object to contain the characteristics of a transformer
  * `DeltaDeltaTransformer`: Delta-delta transformer
  * `DeltaWyeTransformer`: Delta-wye transformer
  * `DeltaZigzagTransformer`: Delta-zigzag transformer
  * `WyeDeltaTransformer`: Wye-delta transformer
  * `WyeWyeTransformer`: Wye-wye transformer
  * `WyeZigzagTransformer`: Wye-zigzag transformer
* Loads:
  * `AdmittanceLoad` a constant impedance star load
  * `ImpedanceLoad`: a constant impedance star load
  * `PowerLoad`: a constant power star load
  * `DeltaAdmittanceLoad`, `DeltaImpedanceLoad`, `DeltaPowerLoad`: the same with delta loads
  * `FlexibleLoad`: a flexible load
  * `FlexibleParameter`: an object containing the parameters of a flexible load.
* Others:
  * `Ground`: a ground element
  * `PotentialRef`: a potential reference element.

Let's use some of these elements to build a network with a voltage source, a simplified line and a constant power load.
 This network is a low voltage network (three-phase +  neutral wire).

In [2]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0

# Create a voltage source
un = 400 / np.sqrt(3)  # V
source_voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, source_voltages=source_voltages)

# Add a new LV bus and a load
load_bus = Bus(id="bus", n=4)
load = PowerLoad(id="load", n=4, bus=load_bus, s=[10e3, 10e3, 10e3])  # W

# Add a LV line between the voltage source and the bus
lc = LineCharacteristics("z", z_line=0.1 * np.eye(4, dtype=complex))
line = SimplifiedLine(id="line", n=4, bus1=vs, bus2=load_bus, line_characteristics=lc, length=2.0)

At this point, all the basic elements of the network have been defined and connected. Note that a ground element is
required for the voltage source as it is a star configuration (`n=4` meaning 4 ports: 3 phases + neutral). The
`PotentialRef` element also defines the potential reference for this network which is mandatory.

Now, everything can be encapsulated in an `ElectricalNetwork` object using the `from_element` constructor. The voltage
 source `vs` is given to this constructor. Every element connected to this voltage source is automatically included
 into the network.

In [3]:
# Create the electrical network
en = ElectricalNetwork.from_element(vs)

## Solving a load flow

An authentification is required. Please contact us at [contact@roseautechnologies.com](mailto:contact@roseautechnologies.com) to get identification.

In [4]:
# Authentication
auth = ("username", "password")

Then, the load flow can be solved by requesting our server.

In [5]:
# Solve the load flow
en.solve_load_flow(auth=auth)

2

It retrieves the number of iterations performed by the Newton-Raphson algorithm to get the results. Here, 2.


## Getting the results

The results are now available in every basic object of the network. For instance, the potentials of the `load_bus` are
 available using the property `potentials` of the object. It retrieves 4 values which are the potentials for phases
 `a`, `b`, `c` and `n` (neutral).

This object also include a unit, Volt. This library uses the Python [Pint](https://pint.readthedocs.io/en/stable/)
third-party library to manipulate units. The unit registry `ureg` and the quantity class `_Q` used in this library
are located in the module `roseau.load_flow.utils` module. They can be used to pass quantities to functions. By
instance, it would have been possible to define the `load` variable using this call:

```python
from roseau.load_flow.utils import Q_

load = PowerLoad(id="load", n=4, bus=load_bus, s=Q_([10, 10, 10], "kVA"))
```

By default, a float is assumed to be in VA.

### Getting results per object

In order to get the potentials of a bus, just call the `potentials` property of buses like that:

In [6]:
load_bus.potentials

Magnitude,[ 2.21928183e+02+7.10061202e-21j -1.10964092e+02-1.92195445e+02j -1.10964092e+02+1.92195445e+02j 2.35458233e-15+1.84740620e-19j]
Units,volt


The currents of the line are available using the `currents` property of the `line` objects. It retrieves two arrays:

* the first is the current oriented from the first bus of the line to the second bus of the line. It contains 4
values: one per phase and the neutral current
* the second is the current oriented from the second bus of the line to the first bus of the line.

Here, the sum of these currents is 0 as we have chosen a simplified line element (only an impedance, without shunt).
The sum would have been non-zeros with a `ShuntLine` element.

In [7]:
line.currents

(array([ 4.50596216e+01-3.55030601e-20j, -2.25298108e+01-3.90227770e+01j,
        -2.25298108e+01+3.90227770e+01j, -1.17729117e-14-9.23703098e-19j]) <Unit('ampere')>,
 array([-4.50596216e+01+3.55030601e-20j,  2.25298108e+01+3.90227770e+01j,
         2.25298108e+01-3.90227770e+01j,  1.17729117e-14+9.23703098e-19j]) <Unit('ampere')>)

For a `FlexibleLoad`, the final power values can be retrieved using the property `powers`.

### Data frame results

The results can also be retrieved for the entire network using some properties of the `ElectricalNetwork` instance. The
first results are the potentials (in Volts) of the buses in a [Pandas](https://pandas.pydata.org/docs/) `DataFrame`.

In [8]:
en.buses_potentials

potential
bus_id phase                            
vs     a      2.309401e+02+0.000000e+00j
       b     -1.154701e+02-2.000000e+02j
       c     -1.154701e+02+2.000000e+02j
       n      0.000000e+00+0.000000e+00j
bus    a      2.219282e+02+7.100612e-21j
       b     -1.109641e+02-1.921954e+02j
       c     -1.109641e+02+1.921954e+02j
       n      2.354582e-15+1.847406e-19j

The second element of the results are the currents of the lines in a [Pandas](https://pandas.pydata.org/docs/)
`DataFrame`.

In [9]:
en.branches_currents

current1                    current2
branch_id phase                                                        
line      a      4.505962e+01-3.550306e-20j -4.505962e+01+3.550306e-20j
          b     -2.252981e+01-3.902278e+01j  2.252981e+01+3.902278e+01j
          c     -2.252981e+01+3.902278e+01j  2.252981e+01-3.902278e+01j
          n     -1.177291e-14-9.237031e-19j  1.177291e-14+9.237031e-19j

The third results are the powers of the flexible loads of the network using `en.loads_powers` method. In this example,
the data frame is empty as there is no flexible loads.

## Updating the network

The network can then be updated. Here, the load constant power values are changed. We create an unbalanced situation
and one can remark that the neutral potential of the bus is not any more close to 0 V.

In [10]:
# Change the load to an unbalanced one and rerun the load flow
load.update_powers([15e3, 0, 0])  # VA
en.solve_load_flow(auth=auth)
load_bus.potentials

Magnitude,[ 216.02252269 +0.j -115.47005384-200.j -115.47005384+200.j 14.91758499 +0.j]
Units,volt
